In [16]:
import os
import json
import random
import string

# Load champion and item data from JSON files
with open("../data/raw/champion_data/champions.json", 'r') as f:
    champions_data = json.load(f)
    champions = {int(champion_info["key"]): champion_name for champion_name, champion_info in champions_data["data"].items()}

with open("../data/raw/item_data/items.json", 'r') as f:
    items_data = json.load(f)
    items = items_data["data"]

# Categorize items based on tags and gold value
damage_items = [item_id for item_id, item_info in items.items() if "Damage" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
health_items = [item_id for item_id, item_info in items.items() if "Health" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
spell_damage_items = [item_id for item_id, item_info in items.items() if "SpellDamage" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
defense_items = [item_id for item_id, item_info in items.items() if ("Armor" in item_info.get("tags", []) or "SpellBlock" in item_info.get("tags", [])) and item_info.get("gold", {}).get("total", 0) > 1500]

# Function to generate random player names
def random_player_name():
    return ''.join(random.choices(string.ascii_letters, k=8))

# Function to generate items based on champion stats
def generate_items(champion_info):
    attack = champion_info["attack"]
    defense = champion_info["defense"]
    magic = champion_info["magic"]

    items = []
    if attack > magic and attack > defense:
        items = random.sample(damage_items, 3) + random.sample(defense_items, 3)
    elif magic > attack and magic > defense:
        items = random.sample(spell_damage_items, 3) + random.sample(defense_items, 3)
    else:
        items = random.sample(health_items, 3) + random.sample(defense_items, 3)

    return items

# Function to generate realistic item purchase times
def realistic_purchase_times():
    # Purchase times in seconds for each item, with realistic ranges
    first_item_time = random.randint(480, 900)  # 8 to 15 minutes
    second_item_time = random.randint(720, 1320)  # 12 to 22 minutes
    third_item_time = random.randint(1200, 1800)  # 20 to 30 minutes
    fourth_item_time = random.randint(1500, 2400)  # 25 to 40 minutes
    fifth_item_time = random.randint(1800, 2700)  # 30 to 45 minutes
    sixth_item_time = random.randint(2100, 3000)  # 35 to 50 minutes
    
    return [
        first_item_time,
        second_item_time,
        third_item_time,
        fourth_item_time,
        fifth_item_time,
        sixth_item_time
    ]

# Positions for each player
POSITIONS = ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]

# Generate dummy match data
def generate_dummy_match_data(num_matches=2000):
    all_matches = []
    match_id_counter = 1

    for _ in range(num_matches):
        match_id = f"DUMMY_{match_id_counter}"
        match_id_counter += 1
        game_duration = random.randint(1200, 3600)  # Random game duration between 20 and 60 mins
        team_ids = [100, 200]  # Team 100 (Blue) and Team 200 (Red)

        # Generate champions for each position
        champions_ids = random.sample(list(champions.keys()), 10)  # Pick 10 random champions
        positions = POSITIONS * 2

        # Generate 10 participants (5 for each team)
        participants = []
        for i in range(10):
            team_id = team_ids[i // 5]
            position = positions[i]
            champion_id = champions_ids[i]
            champion_name = champions[champion_id]
            champion_info = champions_data["data"][champion_name]["info"]

            # Random stats for each player
            kills = random.randint(0, 15)
            deaths = random.randint(0, 15)
            assists = random.randint(0, 20)
            win = True if team_id == 100 else False
            gold_earned = random.randint(5000, 20000)
            total_damage_dealt = random.randint(10000, 50000)
            total_damage_taken = random.randint(5000, 30000)
            total_heal = random.randint(0, 10000)

            items = generate_items(champion_info)
            item_purchase_times = realistic_purchase_times()

            # Find matchup champion
            matchup_champion = champions_ids[i + 5] if i < 5 else champions_ids[i - 5]

            participant = {
                "matchId": match_id,
                "gameDuration": game_duration,
                "championId": champion_id,
                "championName": champion_name,
                "teamId": team_id,
                "individualPosition": position,
                "kills": kills,
                "deaths": deaths,
                "assists": assists,
                "win": win,
                "goldEarned": gold_earned,
                "totalDamageDealt": total_damage_dealt,
                "totalDamageTaken": total_damage_taken,
                "totalHeal": total_heal,
                "matchupChampion": f"Champion_{matchup_champion}",
            }

            # Add items and their purchase times
            for j in range(6):
                participant[f"item_{j}"] = items[j]
                participant[f"item_purchase_time_{j}"] = item_purchase_times[j]

            participants.append(participant)

        all_matches.extend(participants)

    return all_matches

# Generate 100,000 entries of dummy data (10,000 matches with 10 participants each)
all_dummy_data = generate_dummy_match_data(num_matches=2000)

# Save as JSON
output_path = "../data/advanced_dummy_data.json"
with open(output_path, 'w') as f:
    json.dump(all_dummy_data, f, indent=4)

print(f"Dummy match details saved to {output_path}")


Dummy match details saved to ../data/advanced_dummy_data.json


In [18]:
import pandas as pd
import json

# Load dummy match data into a pandas DataFrame
with open("../data/advanced_dummy_data.json", "r") as f:
    match_data = json.load(f)

# Convert the list of matches to a DataFrame
df = pd.DataFrame(match_data)

df.head()

,matchId,gameDuration,championId,championName,teamId,individualPosition,kills,deaths,assists,win,...,item_1,item_purchase_time_1,item_2,item_purchase_time_2,item_3,item_purchase_time_3,item_4,item_purchase_time_4,item_5,item_purchase_time_5
0,DUMMY_1,1959,76,Nidalee,100,TOP,2,9,5,True,...,3165,1046,224633,1651,226662,1661,228002,2201,4401,2565
1,DUMMY_1,1959,498,Xayah,100,JUNGLE,4,0,14,True,...,228006,925,3302,1682,8020,1731,2420,2552,223109,2953
2,DUMMY_1,1959,27,Singed,100,MIDDLE,13,11,2,True,...,6609,855,223084,1609,220002,1954,3102,2014,226333,2341
3,DUMMY_1,1959,131,Diana,100,BOTTOM,4,8,4,True,...,226617,1025,3118,1331,223742,2379,224403,2467,3143,2877
4,DUMMY_1,1959,950,Naafiri,100,UTILITY,1,5,8,True,...,3074,1205,4402,1791,226662,2047,226664,1829,223193,2577
